In [1]:
from google.colab import files
import zipfile
import io
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
import numpy as np
import pandas as pd

# 파일 업로드
uploaded = files.upload()

# ZIP 파일 읽기 및 압축 해제
with zipfile.ZipFile(io.BytesIO(uploaded['train.zip']), 'r') as zip_ref:
    zip_ref.extractall('/content/train')

with zipfile.ZipFile(io.BytesIO(uploaded['test.zip']), 'r') as zip_ref:
    zip_ref.extractall('/content/test')

# Train 및 Test 디렉토리 설정
train_dir = '/content/train'
test_dir = '/content/test'

Saving test.zip to test.zip
Saving train.zip to train.zip


In [16]:
# 훈련 데이터를 위한 이미지 데이터 생성기 (데이터 증강 포함)
train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=40, width_shift_range=0.2, height_shift_range=0.2, shear_range=0.2, zoom_range=0.2, horizontal_flip=True, fill_mode='nearest')

# 테스트 데이터를 위한 이미지 데이터 생성기
test_datagen = ImageDataGenerator(rescale=1./255)

# 훈련 이미지를 배치 단위로 불러오기
train_generator = train_datagen.flow_from_directory(train_dir, target_size=(150, 150), batch_size=32, class_mode='categorical')

# 테스트 이미지를 배치 단위로 불러오기
test_generator = test_datagen.flow_from_directory(test_dir, target_size=(150, 150), batch_size=32, class_mode='categorical', shuffle=False)

# CNN 모델
model = tf.keras.models.Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dropout(0.5),
    Dense(512, activation='relu'),
    Dense(train_generator.num_classes, activation='softmax')
])

# 모델 컴파일
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 모델 훈련
history = model.fit(train_generator, epochs=5, validation_data=test_generator)

# 테스트 데이터로 모델 평가
test_loss, test_accuracy = model.evaluate(test_generator)


Found 395 images belonging to 1 classes.
Found 149 images belonging to 1 classes.


/usr/local/lib/python3.10/dist-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Epoch 1/5


/usr/local/lib/python3.10/dist-packages/tensorflow/python/util/dispatch.py:1260: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(None, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return dispatch_target(*args, **kwargs)


13/13 [==============================] - 31s 2s/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 2/5
13/13 [==============================] - 31s 2s/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 3/5
13/13 [==============================] - 28s 2s/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 4/5
13/13 [==============================] - 28s 2s/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 5/5
5/5 [==============================] - 2s 427ms/step - loss: 0.0000e+00 - accuracy: 1.0000


In [17]:
# 입력 이미지 업로드
uploaded_test_images = files.upload()

# 업로드한 이미지 파일명 가져오기
test_image_name = list(uploaded_test_images.keys())[0]  # 첫 번째 업로드한 이미지 이름

# 이미지 로드 및 사이즈 조정
test_image = load_img(test_image_name, target_size=(150, 150))
test_image = img_to_array(test_image)
test_image = np.expand_dims(test_image, axis=0)

# 클래스 예측
result = model.predict(test_image)
predicted_class = np.argmax(result, axis=1)

# 해당 클래스의 레이블 가져오기
label_map = train_generator.class_indices
predicted_label = list(label_map.keys())[list(label_map.values()).index(predicted_class)]


Saving 10.png to 10.png
1/1 [==============================] - 0s 97ms/step
